In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './prepped_data/'

In [5]:
bdinfo_house_number_df = pd.read_csv(basedir + '표제부_기준년월별_총세대수합.csv')
print(bdinfo_house_number_df.shape)
bdinfo_house_number_df = bdinfo_house_number_df[['지번주소', '기준년월', '관리_건축물대장_PK', '주소별_기준년월_총세대수합',
                                                 '기준년월_최초사용승인일', '주소별_최초생성일자'
                                                ]]
bdinfo_house_number_df.head()

(894005, 6)


,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년월_최초사용승인일,주소별_최초생성일자
0,서울특별시 강남구 개포동 1163-4,201412,11680-100218729,20,20130307.0,20130313
1,서울특별시 강남구 개포동 1163-4,201512,11680-100218729,20,20130307.0,20130313
2,서울특별시 강남구 개포동 1163-4,201612,11680-100218729,20,20130307.0,20130313
3,서울특별시 강남구 개포동 1163-4,201712,11680-100218729,20,20130307.0,20130313
4,서울특별시 강남구 개포동 1163-4,201812,11680-100218729,20,20130307.0,20130313


In [6]:
wbdinfo_house_number_df = pd.read_csv('./prepped_data/총괄표제부_기준년월별_총세대수합.csv')
print(wbdinfo_house_number_df.shape)
wbdinfo_house_number_df.head()

(45376, 6)


,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년월_최초사용승인일,주소별_최초생성일자
0,서울특별시 강남구 개포동 1193-5,201812,11680-100276255,29,20181112.0,20181114
1,서울특별시 강남구 개포동 1193-5,201912,11680-100276255,29,20181112.0,20181114
2,서울특별시 강남구 개포동 1193-5,202011,11680-100276255,29,20181112.0,20181114
3,서울특별시 강남구 개포동 12-0,201412,11680-103,4199,NaN,20120822
4,서울특별시 강남구 개포동 12-0,201512,11680-103,4199,NaN,20120822


In [7]:
concat_df = pd.concat([bdinfo_house_number_df, wbdinfo_house_number_df]).sort_values(['지번주소', '기준년월']).reset_index(drop=True)
print(concat_df.shape)

(939381, 6)


In [8]:
concat_df.head()

,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년월_최초사용승인일,주소별_최초생성일자
0,서울특별시 강남구 개포동 1163-4,201412,11680-100218729,20,20130307.0,20130313
1,서울특별시 강남구 개포동 1163-4,201512,11680-100218729,20,20130307.0,20130313
2,서울특별시 강남구 개포동 1163-4,201612,11680-100218729,20,20130307.0,20130313
3,서울특별시 강남구 개포동 1163-4,201712,11680-100218729,20,20130307.0,20130313
4,서울특별시 강남구 개포동 1163-4,201812,11680-100218729,20,20130307.0,20130313


In [9]:
concat_df['기준년월_최초사용승인일'] = concat_df['기준년월_최초사용승인일'].astype('Int64')

In [10]:
concat_df['주소별_기준년월_총세대수합_최대'] = concat_df[['지번주소', '기준년월', '주소별_기준년월_총세대수합']].groupby(['지번주소', '기준년월']).transform('max')['주소별_기준년월_총세대수합']
concat_df.head()

,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년월_최초사용승인일,주소별_최초생성일자,주소별_기준년월_총세대수합_최대
0,서울특별시 강남구 개포동 1163-4,201412,11680-100218729,20,20130307,20130313,20
1,서울특별시 강남구 개포동 1163-4,201512,11680-100218729,20,20130307,20130313,20
2,서울특별시 강남구 개포동 1163-4,201612,11680-100218729,20,20130307,20130313,20
3,서울특별시 강남구 개포동 1163-4,201712,11680-100218729,20,20130307,20130313,20
4,서울특별시 강남구 개포동 1163-4,201812,11680-100218729,20,20130307,20130313,20


In [11]:
concat_df.drop(columns=['주소별_기준년월_총세대수합'], inplace=True)
concat_df.drop_duplicates(inplace=True)
concat_df['주소별_기준년월_총세대수합_최대'].describe()

count    939381.000000
mean        189.263049
std         621.500722
min           0.000000
25%           7.000000
50%          10.000000
75%          20.000000
max        6920.000000
Name: 주소별_기준년월_총세대수합_최대, dtype: float64

In [12]:
concat_df.shape

(939381, 6)

In [13]:
floor_info_df = pd.read_csv(basedir + 'living_area_per_building.csv')
print(floor_info_df.shape)
floor_info_df.head()

(416654, 4)


,관리_건축물대장_PK,지번주소,도로명주소,주거용연면적
0,11440-3279202,서울특별시 마포구 중동 81-29,NaN,259.38
1,11440-501202,서울특별시 마포구 아현동 694-1,NaN,503.38
2,11440-506902,서울특별시 마포구 아현동 702-5,NaN,857.46
3,11545-642602,서울특별시 금천구 독산동 978-4,NaN,587.17
4,11545-648502,서울특별시 금천구 독산동 182-10,NaN,502.22


In [14]:
floor_info_df.drop(columns=['도로명주소'], inplace=True)

In [15]:
floor_info_df['addr_split_len'] = floor_info_df['지번주소'].apply(lambda x: len(x.split(' ')))
floor_info_df['addr_split_len'].value_counts()

4    413153
5      3264
6       210
7        15
3         8
8         3
9         1
Name: addr_split_len, dtype: int64

In [16]:
len3_df = floor_info_df[floor_info_df['addr_split_len'] == 3]
floor_info_df.drop(len3_df.index, inplace=True)
len6_df = floor_info_df[floor_info_df['addr_split_len'] == 6]
floor_info_df.drop(len6_df.index, inplace=True)
len7_df = floor_info_df[floor_info_df['addr_split_len'] == 7]
floor_info_df.drop(len7_df.index, inplace=True)
len8_df = floor_info_df[floor_info_df['addr_split_len'] == 8]
floor_info_df.drop(len8_df.index, inplace=True)
len9_df = floor_info_df[floor_info_df['addr_split_len'] == 9]
floor_info_df.drop(len9_df.index, inplace=True)

In [17]:
floor_info_df['addr_split_len'].value_counts()

4    413153
5      3264
Name: addr_split_len, dtype: int64

In [18]:
len5_df = floor_info_df[floor_info_df['addr_split_len'] == 5]
len5_df.head()

,관리_건축물대장_PK,지번주소,주거용연면적,addr_split_len
6194,11590-671702,서울특별시 동작구 사당동 산 10-32,423.95,5
16522,11620-1293202,서울특별시 관악구 봉천동 산 101-407,240.96,5
21631,11620-2355902,서울특별시 관악구 봉천동 산 101-420,204.30,5
23486,11620-2110802,서울특별시 관악구 봉천동 산 101-193,247.80,5
23864,11740-983102,서울특별시 강동구 성내동 산 17-4,246.06,5


In [19]:
len5_df['fourth_addr'] = len5_df['지번주소'].apply(lambda x: x.split(' ')[3])
len5_df['fourth_addr'].value_counts()

산               3023
블록               227
블록천왕도시개발사업지구      14
Name: fourth_addr, dtype: int64

In [20]:
not_san_df = len5_df[len5_df['fourth_addr'] != '산']
not_san_df.shape

(241, 5)

In [21]:
floor_info_df.drop(not_san_df.index, inplace=True)

In [22]:
def new_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split(' ')
        return splitted[0] + ' ' + splitted[1] + ' ' + splitted[2] + ' ' + splitted[3] + splitted[4]

In [23]:
len5_df['지번주소'] = len5_df['지번주소'].apply(new_addr)

In [24]:
len5_df = len5_df[len5_df['fourth_addr'] == '산']

In [25]:
floor_info_df.loc[len5_df.index, '지번주소'] = len5_df['지번주소']

In [26]:
floor_info_df.reset_index(drop=True, inplace=True)

In [27]:
len5_df['지번주소'].unique()[:50]

array(['서울특별시 동작구 사당동 산10-32', '서울특별시 관악구 봉천동 산101-407',
       '서울특별시 관악구 봉천동 산101-420', '서울특별시 관악구 봉천동 산101-193',
       '서울특별시 강동구 성내동 산17-4', '서울특별시 관악구 봉천동 산101-419',
       '서울특별시 관악구 봉천동 산101-196', '서울특별시 관악구 봉천동 산101-112',
       '서울특별시 관악구 봉천동 산101-76', '서울특별시 관악구 봉천동 산101-306',
       '서울특별시 관악구 봉천동 산101-336', '서울특별시 관악구 봉천동 산101-410',
       '서울특별시 관악구 봉천동 산101-411', '서울특별시 동작구 노량진동 산12-0',
       '서울특별시 관악구 봉천동 산101-409', '서울특별시 동작구 본동 산13-0',
       '서울특별시 서대문구 홍은동 산11-209', '서울특별시 서대문구 홍은동 산26-112',
       '서울특별시 강서구 화곡동 산172-4', '서울특별시 관악구 봉천동 산101-414',
       '서울특별시 동작구 사당동 산25-2', '서울특별시 구로구 개봉동 산2-29',
       '서울특별시 금천구 시흥동 산173-8', '서울특별시 구로구 개봉동 산2-28',
       '서울특별시 관악구 봉천동 산101-413', '서울특별시 노원구 월계동 산144-1',
       '서울특별시 관악구 봉천동 산101-416', '서울특별시 관악구 봉천동 산101-101',
       '서울특별시 관악구 신림동 산160-35', '서울특별시 관악구 봉천동 산133-15',
       '서울특별시 서대문구 연희동 산22-10', '서울특별시 서대문구 연희동 산118-57',
       '서울특별시 은평구 신사동 산86-11', '서울특별시 관악구 봉천동 산101-111',
       '서울특별시 관악구 봉천동 산101-41

In [38]:
floor_info_df.to_csv('./prepped_data/living_area_per_building_2.csv', index=False)

In [30]:
concat_df.head()

,지번주소,기준년월,관리_건축물대장_PK,기준년월_최초사용승인일,주소별_최초생성일자,주소별_기준년월_총세대수합_최대
0,서울특별시 강남구 개포동 1163-4,201412,11680-100218729,20130307,20130313,20
1,서울특별시 강남구 개포동 1163-4,201512,11680-100218729,20130307,20130313,20
2,서울특별시 강남구 개포동 1163-4,201612,11680-100218729,20130307,20130313,20
3,서울특별시 강남구 개포동 1163-4,201712,11680-100218729,20130307,20130313,20
4,서울특별시 강남구 개포동 1163-4,201812,11680-100218729,20130307,20130313,20


In [31]:
floor_info_df.head()

,관리_건축물대장_PK,지번주소,주거용연면적,addr_split_len
0,11440-3279202,서울특별시 마포구 중동 81-29,259.38,4
1,11440-501202,서울특별시 마포구 아현동 694-1,503.38,4
2,11440-506902,서울특별시 마포구 아현동 702-5,857.46,4
3,11545-642602,서울특별시 금천구 독산동 978-4,587.17,4
4,11545-648502,서울특별시 금천구 독산동 182-10,502.22,4


In [32]:
floor_info_df.shape

(416176, 4)

In [36]:
floor_info_df.drop_duplicates(inplace=True)
floor_info_df.shape

(293183, 4)

In [39]:
floor_info_df['관리_건축물대장_PK'].nunique()

293074

In [41]:
concat_df.shape

(939381, 6)

In [44]:
merge1_df = concat_df.merge(floor_info_df[['관리_건축물대장_PK', '주거용연면적']], on=['관리_건축물대장_PK'])
print(merge1_df.shape)

(864866, 7)
